<a href="https://colab.research.google.com/github/rongxie2023/dlcv/blob/main/Chapter02//home/xr/dlcv/Chapter02/09-case3-softmax_cross_entropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

In [19]:
X = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
X.sum(0), X.sum(0, keepdim=True), X.sum(1, keepdim=True)

(tensor([5., 7., 9.]),
 tensor([[5., 7., 9.]]),
 tensor([[ 6.],
         [15.]]))

### 计算softmax

In [60]:
def softmax(X):
    X_exp = torch.exp(X)  ## ***这里未对数据进行处理，避免exp(X)无穷大的情况
    partition = X_exp.sum(1, keepdim=True)
    return X_exp / partition  # 这里应用了广播机制

In [57]:
# 设置随机种子以获得可重复的结果
torch.manual_seed(0)

X = torch.normal(0, 1, (2, 5))
print(X)
X_prob = softmax(X)
X_prob, X_prob.sum(1)


tensor([[ 1.5410, -0.2934, -2.1788,  0.5684, -1.0845],
        [-1.3986,  0.4033,  0.8380, -0.7193, -0.4033]])


(tensor([[0.6118, 0.0977, 0.0148, 0.2313, 0.0443],
         [0.0474, 0.2873, 0.4437, 0.0935, 0.1282]]),
 tensor([1.0000, 1.0000]))

In [24]:
print(X)
nn.Softmax(dim=1)(X)


tensor([[ 1.5410, -0.2934, -2.1788,  0.5684, -1.0845],
        [-1.3986,  0.4033,  0.8380, -0.7193, -0.4033]])


tensor([[0.6118, 0.0977, 0.0148, 0.2313, 0.0443],
        [0.0474, 0.2873, 0.4437, 0.0935, 0.1282]])

### 2 计算交叉熵

交叉熵采用真实标签的预测概率的负对数似然。 这里我们不使用Python的for循环迭代预测（这往往是低效的）， 而是通过一个运算符选择所有元素。 下面，我们创建一个数据样本y_hat，其中包含2个样本在3个类别的预测概率， 以及它们对应的标签y。 有了y，我们知道在第一个样本中，第一类是正确的预测； 而在第二个样本中，第三类是正确的预测。 然后使用y作为y_hat中概率的索引， 我们选择第一个样本中第一个类的概率和第二个样本中第三个类的概率。

In [37]:
y = torch.tensor([0, 1])
y_hat = torch.tensor([[0.7, 0.2, 0.1], [0.1, 0.8, 0.1]])
y_hat[[0, 1], y]

tensor([0.7000, 0.8000])

In [58]:
def cross_entropy(y_hat, y):
    y_pred = torch.clip(y_hat, 1e-15, 1 - 1e-15)  # 确保预测值在有效范围内，防止 log(0) 导致无穷大
    return - torch.log(y_hat[range(len(y_pred)), y])

print(y_hat,y)
print(cross_entropy(y_hat, y))
print(cross_entropy(softmax(y_hat), y))

tensor([[0.7000, 0.2000, 0.1000],
        [0.1000, 0.8000, 0.1000]]) tensor([0, 1])
tensor([0.3567, 0.2231])
tensor([0.7679, 0.6897])


In [ ]:
#  Cross Entropy 1 =−(1⋅log(0.7)+0⋅log(0.2)+0⋅log(0.1)) => 0.3567
#  Cross Entropy 2 =−(0⋅log(0.1)+1⋅log(0.8)+0⋅log(0.1)) => 0.2231

In [76]:

# 定义交叉熵损失函数
criterion = nn.CrossEntropyLoss()

print(criterion(y_hat, y))   # nn.CrossEntropyLoss 对y_hat进行了softmax操作，并计算每条数据的交叉熵损失，并对整个批量的损失进行平均

# (0.7679+0.6897)/2 => 0.7288

print(criterion(y_hat[0], y[0]))
print(criterion(y_hat[1], y[1]))

### NLLLoss（Negative Log Likelihood Loss）通常用于处理已经过 LogSoftmax 处理的数据
nll_criterion = nn.NLLLoss() 
print("nll",nll_criterion(y_hat, y) )
print(111,y_hat,y[0])
print("nll 0",nll_criterion(y_hat[0], torch.tensor(0)))  #nll也是值越小越相似
print("nll 1",nll_criterion(y_hat[0], torch.tensor(1)))
print("nll 2",nll_criterion(y_hat[0], torch.tensor(2)))

print("nll",nll_criterion(torch.log(softmax(y_hat)), y) )


tensor(0.7288)
tensor(0.7679)
tensor(0.6897)
nll tensor(-0.7500)
111 tensor([[0.7000, 0.2000, 0.1000],
        [0.1000, 0.8000, 0.1000]]) tensor(0)
nll 0 tensor(-0.7000)
nll 1 tensor(-0.2000)
nll 2 tensor(-0.1000)
nll tensor(0.7288)
